In [3]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from custom_dataset import CustomDataset
from rnn_model import RNNModel

# 수정된 PaddingCollate 클래스
class PaddingCollate:
    @staticmethod
    def __call__(batch):
        time = [item['data'][0] for item in batch]
        pin1 = [item['data'][1] for item in batch]
        pin2 = [item['data'][2] for item in batch]

        max_length_pin1 = max(len(seq) for seq in pin1)
        max_length_pin2 = max(len(seq) for seq in pin2)
        max_length = max(max_length_pin1, max_length_pin2)

        padded_pin1 = []
        padded_pin2 = []

        for seq1, seq2 in zip(pin1, pin2):
            # 리스트를 텐서로 변환하여 패딩 수행
            padded_pin1.append(PaddingCollate.pad_sequence(torch.tensor(seq1), max_length))
            padded_pin2.append(PaddingCollate.pad_sequence(torch.tensor(seq2), max_length))

        time_tensor = torch.tensor(time, dtype=torch.float)
        padded_pin1_tensor = torch.stack(padded_pin1)
        padded_pin2_tensor = torch.stack(padded_pin2)

        return time_tensor, padded_pin1_tensor, padded_pin2_tensor

    @staticmethod
    def pad_sequence(sequence, max_length):
        padding_length = max_length - sequence.size(0)
        padded_sequence = torch.nn.functional.pad(sequence, (0, padding_length))
        return padded_sequence


# 저장된 데이터셋 파일 이름
file_name = 'concatenated_dataset'+'.pth'

# 데이터셋 로드
dataset = torch.load(file_name)
print(dataset)

# DataLoader에 로드된 데이터셋 사용
batch_size = 64  # 배치 크기 설정
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=PaddingCollate())

# 학습 반복 횟수 설정
num_epochs = 100

# 모델 생성
input_size = 2  # 입력 차원 설정 (pin1과 pin2)
hidden_size = 64  # 은닉 상태의 크기 설정
output_size = 1  # 출력 차원 설정
num_layers = 2
model = RNNModel(input_size, hidden_size, output_size, num_layers)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(num_epochs):
    total_loss = 0
    for i, (time, pin1, pin2) in enumerate(dataloader):
        # 모델 입력 준비
        inputs = torch.stack([pin1, pin2], dim=2)  # pin1과 pin2를 합쳐서 입력으로 사용
        inputs = inputs.float()  # 데이터 타입을 float으로 변환
        targets = time.float()  # 타겟은 측정 시간으로 설정

        # Forward pass
        outputs = model(inputs)

        # Loss 계산 및 Backpropagation
        loss = criterion(outputs.squeeze(), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Epoch마다 손실 출력
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader)}')

torch.save(model.state_dict(), 'rnn_model.pth')
print("모델 저장이 완료되었습니다.")

Epoch [1/100], Loss: 2.5366011261940002
Epoch [2/100], Loss: 1.4121361076831818
Epoch [3/100], Loss: 0.37228966131806374
Epoch [4/100], Loss: 0.1364179728552699
Epoch [5/100], Loss: 0.2221512384712696


C:\Users\UIT801-\AppData\Local\Temp\ipykernel_122620\3726986710.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_pin1.append(PaddingCollate.pad_sequence(torch.tensor(seq1), max_length))
C:\Users\UIT801-\AppData\Local\Temp\ipykernel_122620\3726986710.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_pin2.append(PaddingCollate.pad_sequence(torch.tensor(seq2), max_length))


Epoch [6/100], Loss: 0.03936552442610264
Epoch [7/100], Loss: 0.058854042552411556
Epoch [8/100], Loss: 0.06978949997574091
Epoch [9/100], Loss: 0.028242580126971006
Epoch [10/100], Loss: 0.022198363672941923
Epoch [11/100], Loss: 0.03181131603196263
Epoch [12/100], Loss: 0.023522928124293685
Epoch [13/100], Loss: 0.01812517666257918
Epoch [14/100], Loss: 0.02087135473266244
Epoch [15/100], Loss: 0.019548729993402958
Epoch [16/100], Loss: 0.01758422702550888
Epoch [17/100], Loss: 0.01775480480864644
Epoch [18/100], Loss: 0.017672873567789793
Epoch [19/100], Loss: 0.016710652504116297
Epoch [20/100], Loss: 0.0167253315448761
Epoch [21/100], Loss: 0.016552595421671867
Epoch [22/100], Loss: 0.016313451109454036
Epoch [23/100], Loss: 0.01648421515710652
Epoch [24/100], Loss: 0.016397927654907107
Epoch [25/100], Loss: 0.016350508201867342
Epoch [26/100], Loss: 0.01616275729611516
Epoch [27/100], Loss: 0.01604279689490795
Epoch [28/100], Loss: 0.015958501491695642
Epoch [29/100], Loss: 0.015

In [8]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from custom_dataset import CustomDataset
from rnn_model import RNNModel

# 수정된 PaddingCollate 클래스
class PaddingCollate:
    @staticmethod
    def __call__(batch):
        time = [item['data'][0] for item in batch]
        pin1 = [item['data'][1] for item in batch]
        pin2 = [item['data'][2] for item in batch]

        max_length_pin1 = max(len(seq) for seq in pin1)
        max_length_pin2 = max(len(seq) for seq in pin2)
        max_length = max(max_length_pin1, max_length_pin2)

        padded_pin1 = []
        padded_pin2 = []

        for seq1, seq2 in zip(pin1, pin2):
            # 리스트를 텐서로 변환하여 패딩 수행
            padded_pin1.append(PaddingCollate.pad_sequence(torch.tensor(seq1), max_length))
            padded_pin2.append(PaddingCollate.pad_sequence(torch.tensor(seq2), max_length))

        time_tensor = torch.tensor(time, dtype=torch.float)
        padded_pin1_tensor = torch.stack(padded_pin1)
        padded_pin2_tensor = torch.stack(padded_pin2)

        return time_tensor, padded_pin1_tensor, padded_pin2_tensor

    @staticmethod
    def pad_sequence(sequence, max_length):
        padding_length = max_length - sequence.size(0)
        padded_sequence = torch.nn.functional.pad(sequence, (0, padding_length))
        return padded_sequence


# 저장된 데이터셋 파일 이름
file_name = 'concatenated_dataset'+'.pth'

# 데이터셋 로드
dataset = torch.load(file_name)
print(dataset)

# DataLoader에 로드된 데이터셋 사용
batch_size = 50  # 배치 크기 설정
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=PaddingCollate())

# 학습 반복 횟수 설정
num_epochs = 100

# 모델 생성
input_size = 2  # 입력 차원 설정 (pin1과 pin2)
hidden_size = 50  # 은닉 상태의 크기 설정
output_size = 1  # 출력 차원 설정
num_layers = 2
model = RNNModel(input_size, hidden_size, output_size, num_layers)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(num_epochs):
    total_loss = 0
    for i, (time, pin1, pin2) in enumerate(dataloader):
        # 모델 입력 준비
        inputs = torch.stack([pin1, pin2], dim=2)  # pin1과 pin2를 합쳐서 입력으로 사용
        inputs = inputs.float()  # 데이터 타입을 float으로 변환
        targets = time.float()  # 타겟은 측정 시간으로 설정

        # Forward pass
        outputs = model(inputs)

        # Loss 계산 및 Backpropagation
        loss = criterion(outputs.squeeze(), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Epoch마다 손실 출력
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader)}')

torch.save(model.state_dict(), 'rnn_model.pth')
print("모델 저장이 완료되었습니다.")

Epoch [1/100], Loss: 1.5436136722564697
Epoch [2/100], Loss: 0.480072882771492
Epoch [3/100], Loss: 0.06180946938693523
Epoch [4/100], Loss: 0.15235181748867035


C:\Users\UIT801-\AppData\Local\Temp\ipykernel_122620\2169376944.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_pin1.append(PaddingCollate.pad_sequence(torch.tensor(seq1), max_length))
C:\Users\UIT801-\AppData\Local\Temp\ipykernel_122620\2169376944.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_pin2.append(PaddingCollate.pad_sequence(torch.tensor(seq2), max_length))


Epoch [5/100], Loss: 0.06093480475246906
Epoch [6/100], Loss: 0.02264699451625347
Epoch [7/100], Loss: 0.040650004893541335
Epoch [8/100], Loss: 0.03085224740207195
Epoch [9/100], Loss: 0.018884951807558538
Epoch [10/100], Loss: 0.021866797283291817
Epoch [11/100], Loss: 0.020743385516107084
Epoch [12/100], Loss: 0.018189794570207595
Epoch [13/100], Loss: 0.01860718633979559
Epoch [14/100], Loss: 0.018354412727057935
Epoch [15/100], Loss: 0.017568081244826318
Epoch [16/100], Loss: 0.01776074431836605
Epoch [17/100], Loss: 0.017426693066954612
Epoch [18/100], Loss: 0.01716557703912258
Epoch [19/100], Loss: 0.017192034982144833
Epoch [20/100], Loss: 0.017048465088009834
Epoch [21/100], Loss: 0.01688509341329336
Epoch [22/100], Loss: 0.016784829273819923
Epoch [23/100], Loss: 0.01680669654160738
Epoch [24/100], Loss: 0.016618044674396516
Epoch [25/100], Loss: 0.016462823562324046
Epoch [26/100], Loss: 0.016418404318392275
Epoch [27/100], Loss: 0.016306815110146998
Epoch [28/100], Loss: 0.